# Generating Synthetic Wind Power Time Series

In [ ]:
import os
import yaml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [ ]:
def load_config(config_path):
    with open(config_path, "r") as file:
        return yaml.safe_load(file)

In [ ]:
config_path = "config.yaml"
config = load_config(config_path)

dir = config['data']['final_data']
params = config['synth']

installed_power = 100
hub_height = 100 
rated_power = 1000
Cp = 0.59
rotor_diameter = 60
alpha = 0.1
cut_in_speed = 3
rated_speed = 12.5
cut_out_speed = 25
z_ref = 10
p_model = 'huang'

In [ ]:
files = os.listdir(dir)
file = files[0]

df = pd.read_csv(os.path.join(dir, file))
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)
#df['timestamp'] = df['timestamp'].dt.tz_localize("UTC").dt.tz_convert("Europe/Berlin")
df.drop(['  QN_x', '  QN_y', '  QN'], axis=1, inplace=True)

In [ ]:
df.info()

### Saturation state

Since not all vapor quantities mix with the non-condensing gas, three states must be distinguished:

- <b>Unsaturated</b>: Only the gas phase, namely the gas-vapor mixture, is present. The partial pressure of the vapor $ p_w $ is lower than the saturation vapor pressure $ p_s $. With the partial pressure of the gas $ p_g $ the total pressure can be determined with $ p = p_g + p_d $.
  
- <b>Saturated</b>: In the gas-vapor mixture the condensation begings. The partial pressure of the vapor is equal to saturation vapor pressure $ p_s = p_w $ . 
  
- <b>Over saturated</b>: There are gas and condensate phases, whereby the gas phase (the gas-vapor mixture) is saturated. The relationships of the saturated state apply to the gas phase.

### Composition of moist air

A distinction is made between absolute and relative humidity, whereby both measure the the H<sub>2</sub>O in the vapor phase.

<b>Absolute humidity</b>: Only with respect to the vapor, fog or ice fog is not considered. The absolute humidity is identical with the partial density of water vapor $ \varrho_d $:

$ \varrho_w = \frac{m_w}{V_{Mi}} = \frac{p_w}{R_{H_20}T}$, 

where 
- $ m_w $ is the mass of the water vapor,
- $ V_{Mi} $ is the volume of the gas mixture,
- $ R_{H_2O} $ is the specific gas constant for water,
- $ T $ is the temperature of the mixture.

In the saturated state $ p_s = p_d $ it is the saturation parital density $ \varrho_s $:

$ \varrho_s = \frac{p_s}{R_{H_20}T}$.

### Relative humidity $ \phi $ 
is defined as:

$ \phi = \frac{\varrho_w}{\varrho_s} = \frac{p_{w}}{p_s} $

with:
- $ \phi $: Relative humiditiy 
- $ p_w $: Partial pressure of water vapor in the air (Pa)
- $ p_s $: Saturation vapor pressure (Pa)

The saturation vapor pressure $ p_s $ is the maximal partial pressur for the water at given temperature $ T $. A higher $ p_s $ is not possible, because the additional water will condense to liquid or disublimate to solid water (ice). 

Relative humidity $ \phi $ can take numerical values in the range $ 0 \leq \phi \leq 1 $. That means:

- $ \phi = 0 $ is dry air,
- $ \phi < 1 $ is unsaturated moist air,
- $ \phi = 1 $ is saturated moist air ($ m_{kon} = 0 $)
- $ \phi = 1 $ is oversaturated moist air ($ m_{kon} > 0 $).


### Get air density from air pressure, temperature and relative humidity

Since we need the density of the air, we need to determine the pressure and gas constant values for the mixture of dry air and water vapor.

The air density is therefore dependent from the partial density of the water vapor $ \varrho_w $ and the partial density of the dry air denoted as $ \varrho_g $, where $ g $ stands for gas.

$ \rho = \varrho_g + \varrho_w = \frac{p_g}{R_gT} + \frac{p_w}{R_{H_2O}T} $

where $ R_g $ is the specific gas constant for dry air.

The specific gas constants are given with:

- $ R_g = 287.05 \frac{J}{kgK} $
- $ R_{H_2O} = 461.5 \frac{J}{kgK} $

The partial water vapor pressure is given by first calculating the saturation vapor pressure (Huang, 2018):

$ p_s = \exp\left(34.4942 - \frac{4924.99}{t + 237.1}\right) \cdot (t + 105)^{1.57} (t > 0°C) $

$ p_s = \exp\left(43.4942 - \frac{6545.8}{t + 278}\right) \cdot (t + 868)^{-2} (t \leq 0°C) $

With the relative humidity the partial water vapor pressure can be determined:

$ p_w = \phi * p_s $

The remaining variable $ p_g $ is calculated by subtracting $ p_g $ from the air pressure $ p $:

$ p_g = p - p_w $


In [ ]:
def get_features(data: pd.DataFrame,
                 params: dict,
                 hub_height: float = 100.0,
                 alpha: float = 0.17,
                 z_ref: float = 10.0,
                 p_model: str = 'improved_magnus') -> pd.Series:
    wind_speed = data[params['v_wind']['param']]
    air_pressure = data[params['pressure']['param']]
    temperature = data[params['temperature']['param']]
    relhum = data[params['relhum']['param']]
    # check if relative humidity is in the range between 0 and 1
    if relhum.max() > 1:
        relhum /= 100
    R_dry = 287.05  # Spezifische Gaskonstante für trockene Luft (J/(kg·K))
    R_w = 461.5  # Spezifische Gaskonstante für Wasserdampf (J/(kg·K))
    temperature_kelvin = temperature + 273.15 
    if p_model == 'huang':
        p_s = np.where(
            temperature > 0,
            np.exp(34.494 - (4924.99 / (temperature + 237.1))) / (temperature + 105) ** 1.57,
            np.exp(43.494 - (6545.8 / (temperature + 278))) / (temperature + 868) ** 2
        )
    elif p_model == 'improved_magnus':
        p_s = np.where(
            temperature > 0,
            610.94 * np.exp((17.625 * temperature) / (temperature + 243.04)),
            611.21 * np.exp((22.587 * temperature) / (temperature + 273.86))
    )
    p_w = relhum * p_s
    p_g = air_pressure - p_w
    rho_g = p_g / (R_dry * temperature_kelvin)
    rho_w = p_w / (R_w * temperature_kelvin)
    rho = rho_g + rho_w
    wind_speed_hub = wind_speed
    wind_speed_hub = wind_speed * (hub_height / z_ref) ** alpha
    return rho, wind_speed_hub


def generate_wind_power(data: pd.DataFrame,
                        rated_power: float = 1000.0,
                        Cp: float = 0.59,
                        rotor_diameter: float = 50.0, 
                        cut_in_speed: float = 3.5, 
                        rated_speed: float = 12.5,
                        cut_out_speed: float = 25.0
                        ) -> pd.Series:
    rho = data['rho']
    wind_speed_hub = data['v_wind_hub']
    rotor_area = np.pi * (rotor_diameter / 2) ** 2
    wind_power = np.where(
        wind_speed_hub < cut_in_speed, 0,
        np.where(
            wind_speed_hub <= rated_speed,
            0.5 * rho * rotor_area * Cp * wind_speed_hub ** 3,
            np.where(
                wind_speed_hub <= cut_out_speed,
                rated_power,
                0
            )
        )
    )
    return pd.Series(wind_power, index=data.index)


def plot_power_and_feature(data: pd.DataFrame,
                           params: dict,
                           day: str, 
                           feature: dict,
                           power: pd.Series,
                           save_fig=False): 
    day = pd.Timestamp(day)
    index_0 = power.index.get_loc(day)
    index_1 = power.index.get_loc(day + pd.Timedelta(days=1))
    feature_specs = params[feature]
    series = data[feature_specs['param']]
    feature_name = feature_specs['name']
    unit = feature_specs['unit']
    date = str(series.index[index_0:index_1][0].date())
    fig, ax1 = plt.subplots(figsize=(10, 6))
    fontsize = 14
    lines = []
    # plot power
    line1, = ax1.plot(
    power[index_0:index_1],
    label="Power Output (W)",
    color="black",
    linewidth=2.0
    )
    lines.append(line1)
    # configure secondary y-axis
    ax1.set_xlabel("Time", fontsize=fontsize)
    ax1.set_ylabel("Power Output (W)", fontsize=fontsize)
    ax1.tick_params(axis='y', labelsize=fontsize)
    ax1.tick_params(axis='x', labelsize=fontsize-2)
    ax2 = ax1.twinx()
    # plot feature
    line, = ax2.plot(
        series[index_0:index_1],
        label=f"{feature_name} {unit}",
        linestyle='--',
        #color='blue',
        linewidth=2.0
    )
    lines.append(line)
    # configure primary y-axis
    ax2.set_ylabel(f"{feature_name} {unit}", fontsize=fontsize)
    ax2.tick_params(axis='y', labelsize=fontsize)
    # Format x-axis to show only hours (HH)
    ax1.xaxis.set_major_locator(mdates.HourLocator(interval=1)) 
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H'))
    ticks = ax1.get_xticks()
    ax1.set_xticks(ticks[1:-1])
    # legend
    lines.append(lines.pop(0))
    labels = [line.get_label() for line in lines]
    ax1.legend(lines, labels, loc="upper left", fontsize=fontsize)
    plt.title(f"{feature_name} and Power Output on {date}", fontsize=fontsize)
    fig.tight_layout()
    #plt.grid(True)
    if save_fig:
        save_path = f'figs/{feature_name}'
        os.makedirs(save_path, exist_ok=True)
        save_file = os.path.join(save_path, f'{date}.png')
        plt.savefig(save_file, dpi=300)
        plt.close()
    else:
        plt.show()

In [ ]:
rho, wind_speed_hub = get_features(data=df,
                                   params=params,
                                   hub_height=hub_height,
                                   alpha=alpha,
                                   z_ref=z_ref,
                                   p_model=p_model)

df['rho'] = rho
df['v_wind_hub'] = wind_speed_hub

power = generate_wind_power(data=df,
                            rated_power=rated_power,
                            Cp=Cp,
                            rotor_diameter=rotor_diameter, 
                            cut_in_speed=cut_in_speed, 
                            rated_speed=rated_speed,
                            cut_out_speed=cut_out_speed
                            )

In [ ]:
day = '2023-08-01'
feature='v_wind_hub'
plot_power_and_feature(data=df,
                        params=params,
                        day=day,
                        feature=feature,
                        power=power)      

In [ ]:
for day in np.unique(df.index.date):
    plot_power_and_feature(data=df,
                           params=params,
                           day=str(day),
                           feature=feature,
                           power=power,
                           save_fig=True) 